In [1]:
%defaultDatasource jdbc:h2:mem:db

# Analisando o Uso de Medicamentos
* Esta tarefa analisará os dados conforme três abordagens: análise do comportamento estatístico; análise de correlação entre pares; análise de correlação de variáveis.
* A partir dos arquivos CSV que estão no diretórios /data/nhanes considere as seguintes tabelas SQL.

In [8]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Medicamento;
DROP TABLE IF EXISTS UsoMedicamento;
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

CREATE TABLE Medicamento (
  Codigo VARCHAR(6) NOT NULL,
  NomeMedicamento VARCHAR(100) NOT NULL,
  PRIMARY KEY(Codigo)
) AS SELECT
    Codigo,
    NomeMedicamento
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE UsoMedicamento (
  IdPessoa VARCHAR(5) NOT NULL,
  CodMedicamento VARCHAR(6) NOT NULL,
  DiasUso INTEGER,
  FOREIGN KEY(IdPessoa)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(CodMedicamento)
    REFERENCES Medicamento(Codigo)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    IdPessoa,
    CodMedicamento,
    DiasUso
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

In [4]:
SELECT * FROM Pessoa;
SELECT * FROM Medicamento;
SELECT * FROM UsoMedicamento;

# I. Análise de comportamento estatístico

## 1) Contabilizando o uso de medicamentos

* Liste o nome de todos os medicamentos e a quantidade de pessoas que usa cada um deles.

In [5]:
SELECT Medicamento.NomeMedicamento, COUNT(*) 
FROM Medicamento, UsoMedicamento 
WHERE Medicamento.Codigo = UsoMedicamento.CodMedicamento
GROUP BY UsoMedicamento.CodMedicamento;

## 2) Medicamento mais usado com VIEW

* Informe o código do medicamento mais usado: fazendo uso de VIEW.

In [20]:
DROP TABLE IF EXISTS Contagem;

CREATE VIEW Contagem AS 
    SELECT Medicamento.Codigo, COUNT(*) Usos
    FROM Medicamento, UsoMedicamento 
    WHERE Medicamento.Codigo = UsoMedicamento.CodMedicamento
    GROUP BY UsoMedicamento.CodMedicamento;
    
SELECT Codigo FROM Contagem ORDER BY Usos DESC LIMIT 1;

d00732

## 3) Medicamento mais usado sem VIEW

* Informe o código do medicamento mais usado: sem fazer uso de VIEW.

In [21]:
SELECT CodMedicamento, Count(*) AS Usos
FROM UsoMedicamento
GROUP BY CodMedicamento
ORDER BY Usos DESC LIMIT 1;

## 4) Nome do medicamento mais usado
* Informe o nome do medicamento mais usado (uso de VIEW é opcional).

In [23]:
DROP TABLE IF EXISTS Contagem;

CREATE VIEW Contagem AS 
    SELECT Medicamento.NomeMedicamento, COUNT(*) Usos
    FROM Medicamento, UsoMedicamento 
    WHERE Medicamento.Codigo = UsoMedicamento.CodMedicamento
    GROUP BY UsoMedicamento.CodMedicamento;
    
SELECT NomeMedicamento FROM Contagem ORDER BY Usos DESC LIMIT 1;

LISINOPRIL

## 5) Contabilizando quanto as pessoas usam de medicamento

* Informe o número médio de uso de medicamento por pessoa.

In [28]:
SELECT AVG(Usos) Media
FROM (
    SELECT COUNT(*) Usos
      FROM UsoMedicamento
      GROUP BY IdPessoa
);

3

## 6) Pessoas que usam mais do que a média

* Liste o id das pessoas que usam mais medicamentos do que a média.

In [36]:
SELECT IdPessoa, COUNT(*) Usos FROM UsoMedicamento GROUP BY IdPessoa HAVING Usos > (SELECT AVG(Usos) Media FROM ( SELECT COUNT(*) Usos FROM UsoMedicamento GROUP BY IdPessoa));

## 7) Análise do uso de medicamento

* Considere que um médico quer fazer responder a seguinte questão: pessoas tendem a usar mais medicamentos conforme ficam mais velhas?
* Escreva uma query que ajude o médico a realizar esta análise.

In [53]:
DROP TABLE IF EXISTS Contagem;

CREATE VIEW Contagem AS 
    SELECT Pessoa.Idade, COUNT(*) Usos
    FROM Pessoa, UsoMedicamento 
    WHERE Pessoa.Id = UsoMedicamento.IdPessoa
    GROUP BY UsoMedicamento.IdPessoa;
    
SELECT DISTINCT * FROM Contagem ORDER BY Contagem.Idade DESC, Contagem.Usos DESC; 

# II. Análise de correlação em pares
## Estudo de ação cruzada entre medicamentos

* Um pesquisador quer estudar o efeito cruzado de usar dois medicamentos simultaneamente. 
* Para isso ele precisará da seguinte sequência de queries.

## 8) Medicamentos tomados em conjunto
* Construa uma VIEW com duas colunas: medicamentoA, medicamentoB
* Nessa VIEW, é criada uma tupla (medicamentoA, medicamentoB) toda vez que uma pessoa toma o medicamento A e também o medicamento B.
* Haverá repetição de tuplas, já que mais de uma pessoa pode tomar o mesmo par de medicamentos.
* Cuidado para não duplicar a mesma informação, ou seja, considerando que uma pessoa tomou o medicamentoX e o medicamentoY, cuidado para não criar duas tuplas: (medicamentoX, medicamentoY) e (medicamentoY, medicamentoX).

In [55]:
DROP TABLE IF EXISTS Conjunto;
CREATE VIEW Conjunto AS
    SELECT UsoMedicamentos1.CodMedicamento Medicamento1, UsoMedicamentos2.CodMedicamento Medicamento2
    FROM UsoMedicamento UsoMedicamentos1, UsoMedicamento UsoMedicamentos2
    WHERE UsoMedicamentos1.IdPessoa = UsoMedicamentos2.IdPessoa AND UsoMedicamentos1.CodMedicamento <> UsoMedicamentos2.CodMedicamento;
    
SELECT DISTINCT * FROM Conjunto ;

## 9) Total de medicamento tomados em conjunto
* Apresente o total de pessoas que toma cada par de medicamentos.

In [56]:
SELECT Medicamento1, Medicamento2, COUNT(*) Quantidade FROM Conjunto GROUP BY Medicamento1, Medicamento2;

# III. Análise de correlação entre variáveis
* Consiste na análise da seguinte questão: como a alteração de uma variável afeta a outra.
## Matriz de análise
* Considere a matriz abaixo.

In [5]:
CREATE TABLE Matriz (
  Id VARCHAR(5) NOT NULL,
  Genero INTEGER,
  Idade INTEGER,
  IdadeAte60 BOOLEAN,
  MaisUmAnoMedicamento BOOLEAN,
  MedicamentosAcimaMedia BOOLEAN,
  PRIMARY KEY(Id)
) AS SELECT
    Id,
    Genero,
    Idade,
    FALSE, FALSE, FALSE
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

SELECT * FROM Matriz;

## 10) Atualização simples da matriz
* Utilize o comando UPDATE para atualizar a coluna IdadeAte60 da tabela colocando verdadeiro para as pessoas que têm até 60 anos de idade.

In [6]:
UPDATE Matriz SET IdadeAte60 = true WHERE Idade < 60;
SELECT * FROM Matriz;

## 11) Atualização vinculada a uma segunda tabela
* Utilize o comando UPDATE para atualizar a coluna MaisUmAnoMedicamento, colocando verdadeiro para aquelas pessoas que usam pelo menos um medicamento há mais de um ano.

In [9]:
DROP TABLE IF EXISTS UsoMaisDeUmAno;
CREATE VIEW UsoMaisDeUmAno AS SELECT DISTINCT IdPessoa FROM UsoMedicamento WHERE DiasUso > 365;
UPDATE Matriz SET MaisUmAnoMedicamento = true WHERE Id IN (SELECT * FROM UsoMaisDeUmAno) ;
SELECT * FROM Matriz;

## 12) Atualização baseada em uma consulta complexa
* Utilize o comando UPDATE para atualizar a coluna MedicamentosAcimaMedia, colocando verdadeiro para aquelas pessoas que usam um número de medicamentos acima da média.

In [10]:
DROP TABLE IF EXISTS Media;
CREATE VIEW Media AS SELECT AVG(Usos) FROM (SELECT COUNT(*) Usos FROM UsoMedicamento GROUP BY IdPessoa);

UPDATE Matriz SET MedicamentosAcimaMedia = TRUE WHERE Id IN (
    SELECT IdPessoa
    FROM UsoMedicamento
    GROUP BY IdPessoa
    HAVING COUNT(*) > (SELECT * FROM Media));
              
SELECT * FROM Matriz;

## 13) Que análise interessante pode ser feita?
* A partir da Matriz criada, proponha uma análise interessante.

### 13a) Descreva a seguir (em Markdown) qual a análise que você propõe.

A partir da análise proposta em um exercício anterior, pode ser analisado se realmente pessoas mais velhas consomem medicamentos acima da média

### 13b) Escreva uma consulta em SQL que fornece informações para a análise proposta.

In [18]:
DROP TABLE IF EXISTS Maior60;
DROP TABLE IF EXISTS Menor60;

CREATE VIEW Maior60 AS SELECT IdadeAte60 FROM Matriz WHERE MedicamentosAcimaMedia = true AND IdadeAte60 = false;
CREATE VIEW Menor60 AS SELECT IdadeAte60 FROM Matriz WHERE MedicamentosAcimaMedia = true AND IdadeAte60 = true;

CREATE VIEW Comparacao1 AS SELECT IdadeAte60, COUNT(*) Usos FROM Menor60 GROUP BY IdadeAte60;
CREATE VIEW Comparacao2 AS SELECT IdadeAte60, COUNT(*) Usos FROM Menor60 GROUP BY IdadeAte60;
SELECT * FROM JOIN Comparacao1, Comparacao2;

org.h2.jdbc.JdbcSQLException:  Cannot drop "MENOR60" because "COMPARACAO" depends on it; SQL statement